install ollama and instructor
```
pip install instructor
```
Ollama download instruction are available here 
https://ollama.com/download

Download the heremes pro model 
Heremes pro is latest model from NousResearch. Somebody already uploaded to ollama model library

```
ollama pull adrienbrault/nous-hermes2pro:Q8_0
```
And then you can use the model in the code below



In [1]:
from openai import OpenAI
from pydantic import BaseModel
import instructor


class UserDetail(BaseModel):
    name: str
    age: int


# enables `response_model` in create call
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

user = client.chat.completions.create(
    model="adrienbrault/nous-hermes2pro:Q8_0",
    messages=[
        {
            "role": "user",
            "content": "Jason is 30 years old",
        }
    ],
    response_model=UserDetail,
)

print(user)
#> name='Jason' age=30

name='Jason' age=30


you can use instructor to generate the desire response model response model is used to validate the response. 



And we can use the same for variety of task to ensure the model is wokring with existing system


In [2]:
### synthatitc data
from typing import Iterable

def generate_fake_users(count: int) -> Iterable[UserDetail]:
    return client.chat.completions.create(
        model="adrienbrault/nous-hermes2pro:Q8_0",
        response_model=Iterable[UserDetail],
        messages=[
            {"role": "user", "content": f"Generate a {count} synthetic users"},
        ],
    )


for user in generate_fake_users(5):
    print(user)

name='John Smith' age=28
name='Mary Johnson' age=35
name='William Anderson' age=41
name='Ava Taylor' age=26
name='Samuel Brown' age=53


In [3]:
from typing import List
class QuestionAnswer(BaseModel):
    question: str
    answer: str
    citations: List[str]
QuestionAnswer.model_json_schema()

{'properties': {'question': {'title': 'Question', 'type': 'string'},
  'answer': {'title': 'Answer', 'type': 'string'},
  'citations': {'items': {'type': 'string'},
   'title': 'Citations',
   'type': 'array'}},
 'required': ['question', 'answer', 'citations'],
 'title': 'QuestionAnswer',
 'type': 'object'}

In [4]:

def ask_ai(question: str, context: str) -> QuestionAnswer:
    return client.chat.completions.create(
        model="adrienbrault/nous-hermes2pro:Q8_0",
        temperature=0.1,
        response_model=QuestionAnswer,
        messages=[
            {
                "role": "system",
                "content": "You are a world class algorithm to answer questions with correct and exact citations.",
            },
            {"role": "user", "content": f"{context}"},
            {"role": "user", "content": f"Question: {question}"},
        ],
        validation_context={"text_chunk": context},
    )

In [5]:
question = "What did the author do during college?"
context = """
My name is Jason Liu, and I grew up in Toronto Canada but I was born in China.
I went to an arts high school but in university I studied Computational Mathematics and physics.
As part of coop I worked at many companies including Stitchfix, Facebook.
I also started the Data Science club at the University of Waterloo and I was the president of the club for 2 years.
"""

response = ask_ai(question, context)

In [6]:
from IPython.display import display
display(response.answer)


'During college, the author studied Computational Mathematics and physics and worked at many companies including Stitchfix and Facebook. They also started the Data Science club at the University of Waterloo and served as its president for 2 years.'

In [7]:
display(response.citations)

['My name is Jason Liu, and I grew up in Toronto Canada but I was born in China.\nI went to an arts high school but in university I studied Computational Mathematics and physics.\nAs part of coop I worked at many companies including Stitchfix, Facebook. \nI also started the Data Science club at the University of Waterloo and I was the president of the club for 2 years.']

In [9]:
question1 = "Where was John born?"
context1 = """
John Doe is a software engineer who was born in New York, USA. 
He studied Computer Science at the Massachusetts Institute of Technology. 
During his studies, he interned at Google and Microsoft. 
He also founded the Artificial Intelligence club at his university and served as its president for three years.
"""


question2 = "What did Emily study in university?"
context2 = """
Emily Smith is a data scientist from London, England. 
She attended the University of Cambridge where she studied Statistics and Machine Learning. 
She interned at IBM and Amazon during her summer breaks. 
Emily was also the head of the Women in Tech society at her university.
"""

question3 = "Which companies did Robert intern at?"
context3 = """
Robert Johnson, originally from Sydney, Australia, is a renowned cybersecurity expert. 
He studied Information Systems at the University of Melbourne. 
Robert interned at several cybersecurity firms including NortonLifeLock and McAfee. 
He was also the leader of the Cybersecurity club at his university.
"""


question4 = "What club did Alice start at her university?"
context4 = """
Alice Williams, a native of Dublin, Ireland, is a successful web developer. 
She studied Software Engineering at Trinity College Dublin. 
Alice interned at several tech companies including Shopify and Squarespace. 
She started the Web Development club at her university and was its president for two years.
"""


question5 = "What did Michael study in high school?"
context5 = """
Michael Brown is a game developer from Tokyo, Japan. 
He attended a specialized high school where he studied Game Design. 
He later attended the University of Tokyo where he studied Computer Science. 
Michael interned at Sony and Nintendo during his university years. 
He also started the Game Developers club at his university.
"""

In [10]:
for question, context in [
    (question1, context1),
    (question2, context2),
    (question3, context3),
    (question4, context4),
    (question5, context5),
]:
    response = ask_ai(question, context)
    display(question)
    display(response.answer)
    display(response.citations)
    print("\n\n")

'Where was John born?'

'John Doe was born in New York, USA.'

['John Doe is a software engineer who was born in New York, USA.']

'What did Emily study in university?'

'Emily studied Statistics and Machine Learning at the University of Cambridge.'

...

'Alice started the Web Development club at her university.'

['Alice Williams, a native of Dublin, Ireland, is a successful web developer. She studied Software Engineering at Trinity College Dublin. Alice interned at several tech companies including Shopify and Squarespace. She started the Web Development club at her university and was its president for two years.']

'What did Michael study in high school?'

'Michael studied Game Design in high school.'

['Michael Brown is a game developer from Tokyo, Japan. He attended a specialized high school where he studied Game Design.']